In [ ]:
import os
import pickle as pkl

import numpy as np
import rf
import rf.imaging
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import obspy
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
# Bring in interactive widgets capability. See https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
import seismic.receiver_fn.rf_util as rf_util
import seismic.receiver_fn.rf_plot_utils as rf_plot_utils

## Choose RF type

In [ ]:
# rf_type = 'ZRT_td'
# rf_type = 'ZRT_fd'
# rf_type = 'LQT_td'
rf_type = 'LQT_fd'

## Choose training station

In [ ]:
training_station = 'BT23'

## Read source file

In [ ]:
src_file = r"..\DATA\OA_rfs_20170911T000036-20181128T230620_{}_rev5_qual.h5".format(rf_type)
oa_all = rf_util.read_h5_rf(src_file, network='OA', station=training_station)

In [ ]:
type(oa_all)

## Select training data

In [ ]:
db = rf_util.rf_to_dict(oa_all)
oa_trainer = db[training_station]
if rf_type[0:3] == 'ZRT':
    prospective_channels = ['HHR', 'BHR']
elif rf_type[0:3] == 'LQT':
    prospective_channels = ['HHQ', 'BHQ']
else:
    prospective_channels = []
# end if
channel = None
for c in prospective_channels:
    if c in oa_trainer:
        channel = c
        break
# end for
print("Selected channel: {}".format(channel))

In [ ]:
len(oa_trainer[channel])

## Add additional statistics for prediction of trace quality

In [ ]:
rf_util.compute_extra_rf_stats(oa_trainer)

## Display ranges of metadata and quality metrics

In [ ]:
def get_metadata_series(traces, field):
    x = [tr.stats.get(field) for tr in traces]
    return x

In [ ]:
# Extract metadata and quality data on all traces for the target channel
channel_data = oa_trainer[channel]

snr = get_metadata_series(channel_data, 'snr')
entropy = get_metadata_series(channel_data, 'entropy')
coherence = get_metadata_series(channel_data, 'max_coherence')
distance = get_metadata_series(channel_data, 'distance')
inclination = get_metadata_series(channel_data, 'inclination')
magnitude = get_metadata_series(channel_data, 'event_magnitude')
depth = get_metadata_series(channel_data, 'event_depth')
rf_group = get_metadata_series(channel_data, 'rf_group')
# Replace no-group group IDs with '-1'
rf_group = [g if g is not None else -1 for g in rf_group]
amax = get_metadata_series(channel_data, 'amax')
# Extra metrics
amp_20pc = get_metadata_series(channel_data, 'amp_20pc')
amp_80pc = get_metadata_series(channel_data, 'amp_80pc')
mean_cplx_amp = get_metadata_series(channel_data, 'mean_cplx_amp')
rms_amp = get_metadata_series(channel_data, 'rms_amp')


In [ ]:
# Generate array of data and labels to use to plot histogram distributions
dist_array = [(snr, "SNR"), (entropy, "Entropy"), (coherence, "Coherence"), (distance, "Distance"),
              (inclination, "Inclination"), (magnitude, "Magnitude"), (amax, "Max amplitude"), (amp_20pc, "Amplitude 20th perc."),
              (amp_80pc, "Amplitude 80th perc."), (mean_cplx_amp, "Mean amplitude"), (rms_amp, "RMS amplitude"), (rf_group, "Group ID")]

In [ ]:
# Plot histograms of metrics
plt.figure(figsize=(20, 12))
plt.subplot(3,4,1)
for i, (data, name) in enumerate(dist_array):
    ax = plt.subplot(3, 4, i + 1)
#     plt.hist(data, bins=20)
    sns.distplot(data, bins=20, ax=ax)
    plt.title(name + " distribution", y=0.88, fontweight='bold')
plt.show()

In [ ]:
# Examine co-plots to look for discriminating variables
df = pd.DataFrame.from_dict({"SNR": snr, "Entropy": entropy, "Coherence": coherence, "Max_amp": amax,
                             "Amp_20pc": amp_20pc, "Amp_80pc": amp_80pc, "RMS_amp": rms_amp, "Mean_amp": mean_cplx_amp,
                             "Magnitude": ">=6", "Distance": ">=60", "Depth": ">=80km",
                             "Inclination": ">=20", "Group_id": rf_group,
                             "Quality": "unknown"})
df.loc[(np.array(magnitude) < 6.0), "Magnitude"] = "<6"
df.loc[(np.array(distance) < 60.0), "Distance"] = "<60"
df.loc[(np.array(inclination) < 20.0), "Inclination"] = "<20"
df.loc[(np.array(depth) < 80.0), "Depth"] = "<80km"

In [ ]:
qual_file = training_station + "_quality_labels_{}.csv".format(rf_type)
if os.path.isfile(qual_file):
    loaded_quality = pd.read_csv(qual_file, index_col=0, header=None)
    df['Quality'] = loaded_quality

### Use interactive widget to manually label the quality of the traces

In [ ]:
print("Quality guide:")
print("'a' = low signal before onset, higher signal after onset with some multiples visible")
print("'b' = signal similar before and after onset, cannot make out multiples with much confidence")
print("Create labels by entering 10 character string of 'a's and 'b's according to quality, ordered from bottom to top trace.")
# Create labels for quality. Note that rf plots are numbered from the bottom up, whereas the Pandas table is displayed ordered from the top down.
quality_updated = False
for i in range(0, len(df), 10):
    existing_qual = df['Quality'].iloc[i:i+10].values
    if not 'unknown' in existing_qual:
        continue
    rf_slice = rf.RFStream(channel_data[i:i+10])
    rf_plot_utils.plot_rf_stack(rf_slice, trace_height=0.4)
    plt.show()
    get_labels = ''
    quit = False
    while len(get_labels) != len(rf_slice):
        get_labels = input("Enter labels: ")
        if get_labels.lower() == 'quit':
            quit = True
            break
        if len(get_labels) != len(rf_slice):
            print("Wrong number of labels, try again!")
    if quit:
        break
    for j, qual in enumerate(get_labels):
        df['Quality'].iloc[i+j] = qual
    quality_updated = True
    display(df.iloc[i:i+10])

if quality_updated:
    df['Quality'].to_csv(qual_file)
else:
    display(df.sample(20, random_state=0))

In [ ]:
# Assign quality category to trace metadata
for i, tr in enumerate(channel_data):
    tr.stats.quality = df['Quality'].iloc[i]

### Plot labelled data to find metrics to discriminate trace quality

In [ ]:
stats_metrics = ["SNR", "Entropy", "Coherence", "Max_amp", "Amp_20pc", "Amp_80pc", "RMS_amp", "Mean_amp"]

In [ ]:
def metrics_pairplot(df, plot_vars, hue_by='Quality', title=''):
    hue_order = None
    if hue_by == 'Quality' or hue_by == 'Prediction':
        hue_order = ['unknown', 'b', 'a'] if 'unknown' in df['Quality'] else ['b', 'a']
    sns.pairplot(df, hue=hue_by, hue_order=hue_order, vars=plot_vars)
    plt.suptitle(title, y=1.01, fontsize=20)
#     plt.show()

In [ ]:
@interact_manual
def _metrics_pairplot(hue_by=['Quality', 'Magnitude', 'Distance', 'Depth', 'Inclination', 'Group_id']):
    metrics_pairplot(df, stats_metrics, hue_by, title="Pairwise quality metrics scatter plot")

In [ ]:
metrics_pairplot(df, plot_vars=stats_metrics, hue_by='Quality')

## Train a neural network classifier to discriminate quality

### Use simple stats for feature vector

In [ ]:
from sklearn.neural_network import MLPClassifier
# from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix

In [ ]:
known_quality_mask = (df['Quality'] != 'unknown')
X = df.loc[known_quality_mask, stats_metrics]
X[np.isnan(X)] = 0
y = df['Quality'].loc[known_quality_mask]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train_transformed = scaler.transform(X_train)

In [ ]:
# This perceptron network has been simplified back to the bare bone so that it corresponds to a linear predictor,
# as higher order complexity and non-linear activation functions gave no improvement in accuracy.
clf_simple = MLPClassifier(solver='lbfgs', alpha=1e-4, max_iter=1000, activation='identity',
                           hidden_layer_sizes=(4,), random_state=3772, tol=1e-4)

In [ ]:
# Run cross-validation to tune hyperparameters
scores = cross_val_score(clf_simple, X_train, y_train, cv=5)
print(scores)
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std()*2))

In [ ]:
# With tuned hyperparameters, train on full training set.
clf_simple.fit(X_train, y_train)
print("Final loss: %0.4f" % clf_simple.loss_)

In [ ]:
final_score = clf_simple.score(X_test, y_test)
print("Final accuracy: %0.3f" % final_score)
# We get decent performance with a trivial network (1 neuron) with trivial activation f(x) = x,
# which means that simply a linear combination of feature vector is sufficient to determine
# classification.

### Apply quality classifier to whole dataset

In [ ]:
X_full = df[stats_metrics]
X_full[np.isnan(X_full)] = 0

In [ ]:
full_prediction = clf_simple.predict(X_full)
confidences = clf_simple.predict_proba(X_full)
confidence_index = np.zeros(full_prediction.shape).astype(np.int)
confidence_index[(full_prediction == 'b')] = 1
df['Prediction'] = full_prediction
df['Confidence'] = confidences[range(confidence_index.size), confidence_index]

In [ ]:
# Show the order of metrics so that weightings can be interpreted in relation to metrics
print(stats_metrics)
# Display the coefficients of the trained classifier
print("Hidden layer:")
print("  weightings: {}".format(clf_simple.coefs_[0].T[0]))
print("        bias: {}".format(clf_simple.intercepts_[0]))
A0 = clf_simple.coefs_[0].T[0]
b0 = clf_simple.intercepts_[0][0]

print("Output layer:")
print("  weightings: {}".format(clf_simple.coefs_[1][0]))
print("        bias: {}".format(clf_simple.intercepts_[1]))
A1 = clf_simple.coefs_[1][0][0]
b1 = clf_simple.intercepts_[1][0]

In [ ]:
# Note: This formulation of prediction metric only applies to single neuron with linear activation function
prediction_metric = A1*(np.matmul(X_full.values, A0) + b0) + b1
df['Prediction metric'] = prediction_metric
df.sample(20, random_state=3772)

In [ ]:
# Plot whole prediction dataset
@interact_manual
def _metrics_pairplot(hue_by=['Prediction', 'Quality']):
    metrics_pairplot(df, stats_metrics, hue_by, title="Pairwise predicted quality scatter plot - full data")

In [ ]:
# Plot prediction just on test dataset
@interact_manual
def _metrics_pairplot(hue_by=['Prediction', 'Quality']):
    metrics_pairplot(df.iloc[sorted(X_test.index)], stats_metrics, hue_by, title="Pairwise predicted quality scatter plot - test data")

In [ ]:
# Display confusion matrix and verify how to compute accuracy from it.
cm = confusion_matrix(df.loc[known_quality_mask, 'Quality'], df.loc[known_quality_mask, 'Prediction'], labels=['b', 'a'])
print(cm)
print(np.sum(cm))
print("Accuracy: %0.3f" % (np.sum(np.diag(cm))/float(np.sum(cm))))

In [ ]:
# Look at how good is the DBSCAN grouping as an indicator of trace quality.
dbscan_group = df['Group_id'].copy()
primary_group_mask = (dbscan_group == 0)
dbscan_group[primary_group_mask] = 'a'
dbscan_group[~primary_group_mask] = 'b'
cm_dbscan = confusion_matrix(df.loc[known_quality_mask, 'Quality'], dbscan_group[known_quality_mask], labels=['b', 'a'])
print(cm_dbscan)
print(np.sum(cm_dbscan))
print("Accuracy: %0.3f" % (np.sum(np.diag(cm_dbscan))/float(np.sum(cm_dbscan))))
# Result here indicates DBSCAN grouping is not a strong predictor of subjective trace quality

In [ ]:
# Look at how good SNR alone is as an indicator of trace quality.
snr_series = df['SNR'].copy()
high_snr_mask = (snr_series >= 2.0)
snr_series[high_snr_mask] = 'a'
snr_series[~high_snr_mask] = 'b'
cm_snr = confusion_matrix(df.loc[known_quality_mask, 'Quality'], snr_series[known_quality_mask], labels=['b', 'a'])
print(cm_snr)
print(np.sum(cm_snr))
print("Accuracy: %0.3f" % (np.sum(np.diag(cm_snr))/float(np.sum(cm_snr))))
# Result here indicates SNR alone is quite a good indicator of quality

In [ ]:
# Look at how good coherence alone is as an indicator of trace quality.
coh_series = df['Coherence'].copy()
high_coh_mask = (coh_series >= 0.25)
coh_series[high_coh_mask] = 'a'
coh_series[~high_coh_mask] = 'b'
cm_coh = confusion_matrix(df.loc[known_quality_mask, 'Quality'], coh_series[known_quality_mask], labels=['b', 'a'])
print(cm_coh)
print(np.sum(cm_coh))
print("Accuracy: %0.3f" % (np.sum(np.diag(cm_coh))/float(np.sum(cm_coh))))

## Persist classifier model to file

In [ ]:
model_file = training_station + "_classifier_{}.pkl".format(rf_type)
with open(model_file, 'wb') as f:
    pkl.dump(clf_simple, f)